In [2]:
#Importar los archivos del Drive
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
#Crear los dataframes
import pandas as pd
#Service_port con los planes
service_port = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/MA5680T_Service_Port.xlsx') #Informacion de los planes
#ONU con los Alias y contratos
onus = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/MA5680T_GPON_ONU.xlsx') #Informacion de los contratos



# Nueva sección

In [4]:
#Ver informacion de los planes de internet
print(service_port)

                                      Name     Alias Connection Type  \
0     25/0_3_10/11/1/Multi-Service VLAN/25     79539        LAN-GPON   
1      25/0_5_5/41/1/Multi-Service VLAN/25  1045 -P6        LAN-GPON   
2      25/0_5_3/88/1/Multi-Service VLAN/25      9985        LAN-GPON   
3      25/0_4_2/2/25/Multi-Service VLAN/25      5629        LAN-GPON   
4      25/0_5_1/16/1/Multi-Service VLAN/25   609 -P5        LAN-GPON   
...                                    ...       ...             ...   
2700   24/0_6_0/36/1/Multi-Service VLAN/24     11973        LAN-GPON   
2701   24/0_3_8/21/1/Multi-Service VLAN/24     92295        LAN-GPON   
2702   24/0_6_0/37/1/Multi-Service VLAN/24      1922        LAN-GPON   
2703   24/0_5_0/65/1/Multi-Service VLAN/24      8166        LAN-GPON   
2704   24/0_6_1/67/1/Multi-Service VLAN/24     11976        LAN-GPON   

                           Interface Information        Service Type  \
0     Frame:0/Slot:3/Port:10/ONTID:11/GEM Port:1  Multi-Service

In [5]:
#Ver informacion de los contratos
print(onus)

     Operation Status Config Status  Frame  Slot  Port  ONU ID  \
0            Activate        Normal      0     3     1      15   
1          Deactivate       Initial      0     3     9       9   
2            Activate       Initial      0     4     0     111   
3            Activate        Normal      0     3     6      38   
4            Activate        Normal      0     3     8      23   
...               ...           ...    ...   ...   ...     ...   
2700         Activate        Normal      0     4     5      53   
2701         Activate        Normal      0     5     0      55   
2702         Activate        Normal      0     5     5      60   
2703         Activate       Initial      0     4     0      83   
2704         Activate       Initial      0     4     0      75   

                                          Name Vendor ID Terminal Type  \
0                                        69629      HWTC      EG8141A5   
1                                         8141        --   

In [6]:
#Para extraer el frame, slot y port y colocarlos en columnas diferentes
def extract_values(row):
    info_parts = row.split('/')
    frame_part = info_parts[0].split(':')[1]
    slot_part = info_parts[1].split(':')[1]
    port_part = info_parts[2].split(':')[1]
    ontid_part = info_parts[3].split(':')[1]
    return frame_part, slot_part, port_part, ontid_part

# Aplicar la función a la columna 'Info' service_port crear columnas separadas en el DataFrame
service_port[['Frame', 'Slot', 'Port', 'ONTID']] = service_port['Interface Information'].apply(extract_values).apply(pd.Series)
print(service_port)

                                      Name     Alias Connection Type  \
0     25/0_3_10/11/1/Multi-Service VLAN/25     79539        LAN-GPON   
1      25/0_5_5/41/1/Multi-Service VLAN/25  1045 -P6        LAN-GPON   
2      25/0_5_3/88/1/Multi-Service VLAN/25      9985        LAN-GPON   
3      25/0_4_2/2/25/Multi-Service VLAN/25      5629        LAN-GPON   
4      25/0_5_1/16/1/Multi-Service VLAN/25   609 -P5        LAN-GPON   
...                                    ...       ...             ...   
2700   24/0_6_0/36/1/Multi-Service VLAN/24     11973        LAN-GPON   
2701   24/0_3_8/21/1/Multi-Service VLAN/24     92295        LAN-GPON   
2702   24/0_6_0/37/1/Multi-Service VLAN/24      1922        LAN-GPON   
2703   24/0_5_0/65/1/Multi-Service VLAN/24      8166        LAN-GPON   
2704   24/0_6_1/67/1/Multi-Service VLAN/24     11976        LAN-GPON   

                           Interface Information        Service Type  \
0     Frame:0/Slot:3/Port:10/ONTID:11/GEM Port:1  Multi-Service

In [7]:
#Pasar a enteros
service_port['Frame'] = service_port['Frame'].astype(int)
service_port['Port'] = service_port['Port'].astype(int)
service_port['Slot'] = service_port['Slot'].astype(int)
service_port['ONTID'] = service_port['ONTID'].astype(int)

In [8]:
# Comparar service_port agregar columna
def agregar_valor(row):
    matching_row = onus[
        (onus['Frame'] == row['Frame']) &
        (onus['Slot'] == row['Slot']) &
        (onus['Port'] == row['Port']) &
        (onus['ONU ID'] == row['ONTID'])
    ]
    if not matching_row.empty:
        return matching_row.iloc[0]['Name']  # Cambiar por la columna que desees
    return None

service_port['Contrato'] = service_port.apply(agregar_valor, axis=1)

print(service_port)

                                      Name     Alias Connection Type  \
0     25/0_3_10/11/1/Multi-Service VLAN/25     79539        LAN-GPON   
1      25/0_5_5/41/1/Multi-Service VLAN/25  1045 -P6        LAN-GPON   
2      25/0_5_3/88/1/Multi-Service VLAN/25      9985        LAN-GPON   
3      25/0_4_2/2/25/Multi-Service VLAN/25      5629        LAN-GPON   
4      25/0_5_1/16/1/Multi-Service VLAN/25   609 -P5        LAN-GPON   
...                                    ...       ...             ...   
2700   24/0_6_0/36/1/Multi-Service VLAN/24     11973        LAN-GPON   
2701   24/0_3_8/21/1/Multi-Service VLAN/24     92295        LAN-GPON   
2702   24/0_6_0/37/1/Multi-Service VLAN/24      1922        LAN-GPON   
2703   24/0_5_0/65/1/Multi-Service VLAN/24      8166        LAN-GPON   
2704   24/0_6_1/67/1/Multi-Service VLAN/24     11976        LAN-GPON   

                           Interface Information        Service Type  \
0     Frame:0/Slot:3/Port:10/ONTID:11/GEM Port:1  Multi-Service

In [9]:
# Comparar service_port agregar columna
def agregar_valor(row):
    matching_row = onus[
        (onus['Frame'] == row['Frame']) &
        (onus['Slot'] == row['Slot']) &
        (onus['Port'] == row['Port']) &
        (onus['ONU ID'] == row['ONTID'])
    ]
    if not matching_row.empty:
        return matching_row.iloc[0]['SN']  # Cambiar por la columna que desees
    return None

service_port['Serial'] = service_port.apply(agregar_valor, axis=1)

print(service_port)

                                      Name     Alias Connection Type  \
0     25/0_3_10/11/1/Multi-Service VLAN/25     79539        LAN-GPON   
1      25/0_5_5/41/1/Multi-Service VLAN/25  1045 -P6        LAN-GPON   
2      25/0_5_3/88/1/Multi-Service VLAN/25      9985        LAN-GPON   
3      25/0_4_2/2/25/Multi-Service VLAN/25      5629        LAN-GPON   
4      25/0_5_1/16/1/Multi-Service VLAN/25   609 -P5        LAN-GPON   
...                                    ...       ...             ...   
2700   24/0_6_0/36/1/Multi-Service VLAN/24     11973        LAN-GPON   
2701   24/0_3_8/21/1/Multi-Service VLAN/24     92295        LAN-GPON   
2702   24/0_6_0/37/1/Multi-Service VLAN/24      1922        LAN-GPON   
2703   24/0_5_0/65/1/Multi-Service VLAN/24      8166        LAN-GPON   
2704   24/0_6_1/67/1/Multi-Service VLAN/24     11976        LAN-GPON   

                           Interface Information        Service Type  \
0     Frame:0/Slot:3/Port:10/ONTID:11/GEM Port:1  Multi-Service

In [10]:
valores_repetidos = service_port[service_port.duplicated('Contrato')]

print("Valores repetidos:")
print(valores_repetidos)

Valores repetidos:
Empty DataFrame
Columns: [Name, Alias, Connection Type, Interface Information, Service Type, Service Para, Upstream Traffic Profile, Downstream Traffic Profile, VLAN ID, Inner VLAN ID, Max. Learnable MAC Addresses, Frame, Slot, Port, ONTID, Contrato, Serial]
Index: []


In [11]:
#Para crear el excel
service_port.to_excel('Planes_GPON_OLT_Principal.xlsx')
file_path = '/content/Planes_GPON_OLT_Principal.xlsx'


In [12]:
#Crea una ruta en drive y mueve el archivo a esa ruta
excel_dest_path = '/content/drive/MyDrive/Colab Notebooks/Planes_GPON_OLT_Principal.xlsx'
!mv "$file_path" "$excel_dest_path"